In [1]:
import urllib
import json
import pandas as pd
import numpy as np
import warnings
import pickle
from datetime import datetime
from datetime import timedelta
import statistics
warnings.filterwarnings('ignore')

In [83]:
PICKLE_PATH = '../augmented_datasets/hopkins_conf_augmented0304.pkl'
RESULT_PATH = '../augmented_datasets/hopkins_conf_gf0304.pkl'

###### Open pickle file

In [3]:
hopkins_confirmed = pd.DataFrame()
hpcon_rates=pd.DataFrame()

with open(PICKLE_PATH, 'rb') as file:
    hopkins_confirmed = pickle.load(file)
    
hopkins_confirmed
hopkins_confirmed.insert(7,column='last relevant date',value=' ')

hopkins_confirmed.insert(7,column='first_7',value=np.nan)  # first 7 days after 5% inf croseed (if exist)

Province_State  \
coordinate           information                                  
(-41.4545, 145.9707) data                              Tasmania   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(-40.9006, 174.886)  data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(-38.4161, -63.6167) data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(-37.8136, 144.9631) data                              Victoria   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(-35.6751, -71.543)  data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(-35.4735, 149.0124) data          Australian Capital Territory   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
...                                                         ...   
(63.0, 16.0)         data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(64.0, 26.0)         data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(64.2823, -135.0)    data                                 Yukon   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(64.8255, -124.8457) data                 Northwest Territories   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(64.9631, -19.0208)  data                                   NaN   
                     avg_d_RH                               NaN   
                     avg_d_precip                           NaN   
                     avg_d_tmp                              NaN   
                     avg_d_wind                             NaN   
(71.7069, -42.6043)  data                             Greenland   
                     avg_d_RH                               NaN   
                     avg_d_pr

In [4]:
datarows = [row for row in hopkins_confirmed.iterrows()  if row[0][1]=='data']

### Obtain a list of dates (string of columns names which are dates , assumed to be a continouos sequence in the DF)

cur=0

for name in hopkins_confirmed.columns :
    if '2020' in name and '/' in name: # this is a date!
        datefound=True
        break
    cur +=1  
if datefound == False:
    print ('date not found in DF columns')
else:
    print ('first date is',name, 'offset',cur )
    firstdate=name
    firstdate_offset=cur 
    
rvrscur=len(hopkins_confirmed.columns)

for name in hopkins_confirmed.columns[::-1]:                #this loop will allow columns which are not dates right of dates
    if '2020' in name and '/' in name: # this is a date!
        break # this is  a date...
    else:
        rvrscur -=1
print ('rvrscur col',hopkins_confirmed.columns[rvrscur-1])
valid_dates=hopkins_confirmed.columns[cur:rvrscur]         # valid dates - all the dates in the index

print ('dates',valid_dates)
print ('last',valid_dates[-1])
 

for row in datarows:
    #print (row[0],row[0][1])
    ratios=[0]
    totals=[]
    actual_dates=[]
    
    for cursor in range (len(valid_dates)-1):
        p=row[1][valid_dates[cursor]]
        q=row[1][valid_dates[cursor+1]  ]
        actual_dates.append(valid_dates[cursor])
        if p > 0:
            ratios.append (round(q/p - 1,4 ))
            totals.append (p)
        else:
            if q>0:
                ratios.append (4)           # the first gr is set to 4 (5 times ...) and will most often get removed due to
                                            # totals less then 5 pct
            else: ratios.append(0)          #  0 growth between two zeros...
            totals.append(0)
            
    #ratios=[0]+ratios  # so that each growth rate 'sits' under its 'total' and is the growth between this and the 
                       # previous
    totals.append(row[1][valid_dates[-1]]) # add the last total to list
    actual_dates.append(valid_dates[-1])
           
    print ('==========')        
    print  ('new',row[1]['Province_State'],row[1][ 'Country_Region'])
    print ('totals',totals)
    print ('ratios',ratios)
       
    if max(totals)<20:
        print ('SKIP')
        continue    ###  if no day with 20 or more, skip this region,  small numbers 
    print ('NO SKIP')
        
   
    print (row[1]['Province_State'],row[1][ 'Country_Region'],ratios,totals)
    print ('lens',len(ratios),len(totals))
    maxt=max(totals)
    few=maxt/20
    print (row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    # here , find the 5% seperator
    few_seperator = -1
    for cursor in range (len(totals)-1):
        if  (totals[cursor] <=few ) and (totals [cursor+1]>=few):
            few_seperator=cursor
            break
    if few_seperator >=0 :
        t_sample=totals[cursor+1:]
        t_ratios=ratios[cursor+1:]
        actual_dates=actual_dates[cursor+1:]
        five_pct_name=actual_dates[0]
        
        print ('five pct', five_pct_name) 
    else:
        five_pct_name=actual_dates[0]  # this is the complete actual_dates
        print ('five pct not found',five_pct_name )
 
    #   find if there is a sequence of growth rate lower than 2% per day (from the end )  remove it if found
    
    for cur in range (len(t_ratios)):
        maxright=max(t_ratios[cur:])
        if   (maxright>0.02):
            continue
        else:
            print ('maxright ,cur, len ratios, rightside',maxright,cur,len(t_ratios),t_ratios[cur:],t_sample[cur:] )
            break
    
    
    last_relevant_sample=actual_dates[ -1]  # overriden if cut   
    if cur < len(t_ratios)-1:
     
        print ('cut right side len befor , len after',len(t_ratios),cur+1)
        
        t_ratios=t_ratios [:cur+1]
        t_sample=t_sample [:cur+1]
        actual_dates=actual_dates[:cur+1]
        last_relevant_sample=actual_dates[ -1]
        
    else: print ('nothing was cut')
            
    
    sorted_sample=t_sample.sort()
    
    
    print ('sample size',len(t_sample),t_sample,t_ratios)
    
    med=statistics.median(t_ratios)
    
    lower=np.percentile(t_ratios,25)
    upper=np.percentile(t_ratios,75)
    
    print ('lower, median, upper',lower,med,upper)
    
    colslen=len(hopkins_confirmed)
    daycount=0
    sevendays=[]
    
    for smpl in t_sample:
        if smpl< few : continue
        else:
            daycount +=1
            if daycount==8 : break
            else: sevendays.append(smpl)
    if daycount==8:
        if sevendays[0]>0:
            
            seven_days_growth=(sevendays[-1]/sevendays[0])-1
    
        else:
            daycount=0
    else: daycount=0
     
    
    coord=row [0][0]
    if daycount>0:
        hopkins_confirmed.loc[coord, 'first_7']['data'] = seven_days_growth
        print ( 'Debug 7 ', seven_days_growth)
        
    hopkins_confirmed.loc[coord, '5%_Date']['data'] = five_pct_name
    hopkins_confirmed.loc[coord, 'last relevant date']['data']=last_relevant_sample
    hopkins_confirmed.loc[coord, 'GF_Q1']['data']=lower
    hopkins_confirmed.loc[coord, 'GF_Q2']['data']=med
    hopkins_confirmed.loc[coord, 'GF_Q3']['data']=upper
    print ('Debug 0 ',hopkins_confirmed.loc[coord,'GF_Q1']['data'], hopkins_confirmed.loc[coord,'GF_Q3']['data'])
    
    print ('DEBUG1' , row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    print ('DEBUG2' , lower, med, upper,five_pct_name,len(actual_dates),len(t_ratios))
    print ('DEBUG3', t_ratios)
    print ('DEBUG4', t_sample)
    print ('DEBUG5',hopkins_confirmed.loc[(row[0][0])])


first date is 1/22/2020 offset 16
rvrscur col 3/27/2020
dates Index(['1/22/2020', '1/23/2020', '1/24/2020', '1/25/2020', '1/26/2020',
       '1/27/2020', '1/28/2020', '1/29/2020', '1/30/2020', '1/31/2020',
       '2/1/2020', '2/2/2020', '2/3/2020', '2/4/2020', '2/5/2020', '2/6/2020',
       '2/7/2020', '2/8/2020', '2/9/2020', '2/10/2020', '2/11/2020',
       '2/12/2020', '2/13/2020', '2/14/2020', '2/15/2020', '2/16/2020',
       '2/17/2020', '2/18/2020', '2/19/2020', '2/20/2020', '2/21/2020',
       '2/22/2020', '2/23/2020', '2/24/2020', '2/25/2020', '2/26/2020',
       '2/27/2020', '2/28/2020', '2/29/2020', '3/1/2020', '3/2/2020',
       '3/3/2020', '3/4/2020', '3/5/2020', '3/6/2020', '3/7/2020', '3/8/2020',
       '3/9/2020', '3/10/2020', '3/11/2020', '3/12/2020', '3/13/2020',
       '3/14/2020', '3/15/2020', '3/16/2020', '3/17/2020', '3/18/2020',
       '3/19/2020', '3/20/2020', '3/21/2020', '3/22/2020', '3/23/2020',
       '3/24/2020', '3/25/2020', '3/26/2020', '3/27/2020'],
      

[5 rows x 82 columns]
new Victoria Australia
totals [0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 7.0, 9.0, 9.0, 10.0, 10.0, 10.0, 11.0, 11.0, 15.0, 18.0, 21.0, 21.0, 36.0, 49.0, 57.0, 71.0, 94.0, 121.0, 121.0, 121.0, 229.0, 355.0, 355.0, 411.0, 466.0, 520.0, 574.0]
ratios [0, 0, 0, 0, 4, 0.0, 0.0, 0.0, 1.0, 0.5, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.0, 0.2857, 0.0, 0.1111, 0.0, 0.0, 0.1, 0.0, 0.3636, 0.2, 0.1667, 0.0, 0.7143, 0.3611, 0.1633, 0.2456, 0.3239, 0.2872, 0.0, 0.0, 0.8926, 0.5502, 0.0, 0.1577, 0.1338, 0.1159, 0.1038]
NO SKIP
Victoria Australia [0, 0, 0, 0, 4, 0.0, 0.0, 0.0, 1.0, 0.5, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Debug 0  0.11435000000000001 0.3612
DEBUG1 South Australia Australia 257.0 12.85
DEBUG2 0.11435000000000001 0.2687 0.3612 3/13/2020 15 15
DEBUG3 [0.7778, 0.1875, 0.0526, 0.45, 0.0, 0.2759, 0.1351, 0.1905, 0.34, 0.4925, 0.34, 0.2687, 0.0, 0.3824, 0.0936]
DEBUG4 [16.0, 19.0, 20.0, 29.0, 29.0, 37.0, 42.0, 50.0, 67.0, 100.0, 134.0, 170.0, 170.0, 235.0, 257.0]
DEBUG5                Province_State Country_Region  avg_m_tmp  avg_m_RH  \
information                                                         
data          South Australia      Australia   20.39697      57.5   
avg_d_RH                  NaN            NaN        NaN       NaN   
avg_d_precip              NaN            NaN        NaN       NaN   
avg_d_tmp                 NaN            NaN        NaN       NaN   
avg_d_wind                NaN            NaN        NaN       NaN   

              avg_m_precip  avg_m_wind  Max_Cases  first_7 last relevant date  \
information                                                           

Debug 0  0.12245 0.5082
DEBUG1 nan Uruguay 238.0 11.9
DEBUG2 0.12245 0.1702 0.5082 3/17/2020 11 11
DEBUG3 [2.625, 0.7241, 0.58, 0.1899, 0.1702, 0.4364, 0.0253, 0.0, 0.1667, 0.1481, 0.0968]
DEBUG4 [29.0, 50.0, 79.0, 94.0, 110.0, 158.0, 162.0, 162.0, 189.0, 217.0, 238.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN        Uruguay  23.510606  68.030303   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases   first_7  \
information                                                   
data                   0.0   14.118182      238.0  4.586207   
avg_d_RH               NaN         NaN        NaN       N

Debug 0  0.20879999999999999 0.35455000000000003
DEBUG1 nan South Africa 1170.0 58.5
DEBUG2 0.20879999999999999 0.28645 0.35455000000000003 3/16/2020 12 12
DEBUG3 [0.2157, 0.0, 0.871, 0.2931, 0.3467, 0.1881, 0.1417, 0.4672, 0.3781, 0.2798, 0.3075, 0.2621]
DEBUG4 [62.0, 62.0, 116.0, 150.0, 202.0, 240.0, 274.0, 402.0, 554.0, 709.0, 927.0, 1170.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN   South Africa  23.168182  59.318182   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases   first_7  \
information                                                   
data                   0.0   12.681818     

...

lower, median, upper 0.0143 0.0833 0.4048
Debug 7  3.25
Debug 0  0.0143 0.4048
DEBUG1 Manitoba Canada 39.0 1.95
DEBUG2 0.0143 0.0833 0.4048 3/13/2020 15 15
DEBUG3 [4, 0.0, 0.0, 0.75, 0.1429, 0.875, 0.1333, 0.0, 0.0588, 0.1111, 0.0, 0.05, 0.6667, 0.0286, 0.0833]
DEBUG4 [4.0, 4.0, 4.0, 7.0, 8.0, 15.0, 17.0, 17.0, 18.0, 20.0, 20.0, 21.0, 35.0, 36.0, 39.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data               Manitoba         Canada -13.268182  78.757576   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases  first_7 last relevant date  \
information                                                                     

DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN      Lithuania   3.042424  74.454545   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases  first_7 last relevant date  \
information                                                                     
data                   0.0        14.2      358.0     6.16          3/27/2020   
avg_d_RH               NaN         NaN        NaN      NaN                      
avg_d_precip           NaN         NaN        NaN      NaN                      
avg_d_tmp              NaN         NaN        NaN      NaN                      
avg_d_wind             NaN    

Debug 0  0.06709999999999999 0.26525
DEBUG1 nan Estonia 575.0 28.75
DEBUG2 0.06709999999999999 0.0949 0.26525 3/13/2020 15 15
DEBUG3 [3.9375, 0.4557, 0.487, 0.1988, 0.0976, 0.1467, 0.0349, 0.0599, 0.0813, 0.0654, 0.0798, 0.0483, 0.0949, 0.3317, 0.0688]
DEBUG4 [79.0, 115.0, 171.0, 205.0, 225.0, 258.0, 267.0, 283.0, 306.0, 326.0, 352.0, 369.0, 404.0, 538.0, 575.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN        Estonia   3.162121  80.727273   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases   first_7  \
information                                                   
data                   0.

Debug 0  0.0934 0.14565
DEBUG1 nan Norway 3755.0 187.75
DEBUG2 0.0934 0.1066 0.14565 3/9/2020 19 19
DEBUG3 [0.1648, 0.9512, 0.495, 0.1739, 0.4188, 0.0944, 0.1202, 0.0917, 0.0975, 0.0595, 0.1265, 0.0962, 0.1066, 0.1261, 0.099, 0.0923, 0.0772, 0.0924, 0.1146]
DEBUG4 [205.0, 400.0, 598.0, 702.0, 996.0, 1090.0, 1221.0, 1333.0, 1463.0, 1550.0, 1746.0, 1914.0, 2118.0, 2385.0, 2621.0, 2863.0, 3084.0, 3369.0, 3755.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN         Norway  -0.006061  82.530303   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

              avg_m_precip  avg_m_wind  Max_Cases   first_7  \
information                             

lower, median, upper 0.0793 0.1243 0.2217
Debug 7  4.055900621118012
Debug 0  0.0793 0.2217
DEBUG1 nan Sweden 3069.0 153.45
DEBUG2 0.0793 0.1243 0.2217 3/7/2020 21 21
DEBUG3 [0.5941, 0.2609, 0.2217, 0.4315, 0.4085, 0.198, 0.3589, 0.1806, 0.0635, 0.0793, 0.0789, 0.0748, 0.1251, 0.139, 0.0757, 0.097, 0.0579, 0.1173, 0.105, 0.1243, 0.0806]
DEBUG4 [161.0, 203.0, 248.0, 355.0, 500.0, 599.0, 814.0, 961.0, 1022.0, 1103.0, 1190.0, 1279.0, 1439.0, 1639.0, 1763.0, 1934.0, 2046.0, 2286.0, 2526.0, 2840.0, 3069.0]
DEBUG5              Province_State Country_Region  avg_m_tmp   avg_m_RH  \
information                                                        
data                    NaN         Sweden  -1.584848  79.212121   
avg_d_RH                NaN            NaN        NaN        NaN   
avg_d_precip            NaN            NaN        NaN        NaN   
avg_d_tmp               NaN            NaN        NaN        NaN   
avg_d_wind              NaN            NaN        NaN        NaN   

         

In [5]:
look=hopkins_confirmed[[ 'Province_State',  'Country_Region','first_7']]
for row in look.iterrows() :
    if row[0][1]=='data' : 
         print (row[1][0:3])

Province_State     Tasmania
Country_Region    Australia
first_7             1.33333
Name: ((-41.4545, 145.9707), data), dtype: object
Province_State            NaN
Country_Region    New Zealand
first_7                  6.75
Name: ((-40.9006, 174.886), data), dtype: object
Province_State          NaN
Country_Region    Argentina
first_7             2.12903
Name: ((-38.4161, -63.6167), data), dtype: object
Province_State     Victoria
Country_Region    Australia
first_7             2.36111
Name: ((-37.8136, 144.9631), data), dtype: object
Province_State        NaN
Country_Region      Chile
first_7           3.07742
Name: ((-35.6751, -71.543), data), dtype: object
Province_State    Australian Capital Territory
Country_Region                       Australia
first_7                                   8.75
Name: ((-35.4735, 149.0124), data), dtype: object
Province_State    South Australia
Country_Region          Australia
first_7                     1.625
Name: ((-34.9285, 138.6007), data), dty

Name: ((29.5, 47.75), data), dtype: object
Province_State    Chongqing
Country_Region        China
first_7             2.70175
Name: ((30.0572, 107.874), data), dtype: object
Province_State         NaN
Country_Region    Pakistan
first_7            4.70588
Name: ((30.3753, 69.3451), data), dtype: object
Province_State    Sichuan
Country_Region      China
first_7           5.32143
Name: ((30.6171, 102.7103), data), dtype: object
Province_State      Hubei
Country_Region      China
first_7           2.80473
Name: ((30.9756, 112.2707), data), dtype: object
Province_State       NaN
Country_Region    Israel
first_7           2.4129
Name: ((31.0, 35.0), data), dtype: object
Province_State    Shanghai
Country_Region       China
first_7            3.09091
Name: ((31.201999999999998, 121.4491), data), dtype: object
Province_State        NaN
Country_Region     Jordan
first_7           5.58824
Name: ((31.24, 36.51), data), dtype: object
Province_State    Tibet
Country_Region    China
first_7       

###### Recalculate interval climate values for between 5% and last_relevant
###### Remove countries with less then 20 confirmed cases

In [79]:
coords = set(hopkins_confirmed.index.get_level_values(0).tolist())
for coord in coords:
    if hopkins_confirmed.loc[coord]['Max_Cases']['data'] < 20:
        hopkins_confirmed.drop(coord, level=0, inplace=True)
        continue
    try:
        last_rel_date = hopkins_confirmed.loc[coord]['last relevant date']['data']
        five_prcnt_date = hopkins_confirmed.loc[coord]['5%_Date']['data']

        five_prct_column = hopkins_confirmed.columns.get_loc(five_prcnt_date)
        last_rel_column = hopkins_confirmed.columns.get_loc(last_rel_date)
        interval = hopkins_confirmed[hopkins_confirmed.columns[five_prct_column:last_rel_column]]

        hopkins_confirmed.loc[coord, 'avg_interval_tmp']['data'] = interval.loc[coord].loc['avg_d_tmp'].mean()
        hopkins_confirmed.loc[coord, 'avg_interval_RH']['data'] = interval.loc[coord].loc['avg_d_RH'].mean()
    except KeyError as e:
        print('key error, {0}'.format(coord))

###### Save file as pickle

In [84]:
with open(RESULT_PATH, 'wb') as file:
    pickle.dump(hopkins_confirmed, file)